In [1]:
!wget https://github.com/luisgasco/ntic_master_datos/raw/main/datasets/tripadvisor_reviews.tsv
# Se lee con el comando dataset =pd.read_csv("tripadvisor_reviews.tsv", sep="\t")
# Recordar descargar los modelos de spacy y nltk necesarios para la resolución del ejercicio

--2021-07-22 18:04:37--  https://github.com/luisgasco/ntic_master_datos/raw/main/datasets/tripadvisor_reviews.tsv
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/luisgasco/ntic_master_datos/main/datasets/tripadvisor_reviews.tsv [following]
--2021-07-22 18:04:37--  https://raw.githubusercontent.com/luisgasco/ntic_master_datos/main/datasets/tripadvisor_reviews.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14925041 (14M) [text/plain]
Saving to: ‘tripadvisor_reviews.tsv’

tripadvisor_reviews 100%[===================>]  14.23M  --.-KB/s    in 0.1s    

2021-07-22 18:04:38 (104 MB/s) - ‘tripadvi

In [2]:
!pip install spacy
!python -m spacy download es_core_news_sm
# Instalamos nltk
!pip install nltk

     |████████████████████████████████| 16.2 MB 14.1 MB/s 
  Created wheel for es-core-news-sm: filename=es_core_news_sm-2.2.5-py3-none-any.whl size=16172932 sha256=7d13a0417944529147bc6113413b742cdb2e5905384a55494a5e4a5674cc8288
  Stored in directory: /tmp/pip-ephem-wheel-cache-fq_t_fu7/wheels/21/8d/a9/6c1a2809c55dd22cd9644ae503a52ba6206b04aa57ba83a3d8
Successfully built es-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_sm')


# Ejercicio 2 (35%)

El objetivo de este ejercicio es comprobar los conocimientos adquiridos por el alumno en temas relacionados a las transformación de textos utilizando la técnica de TFIDF.

- **Apartado 1**: Preprocesar los textos del corpus con las funciones de preprocesado creadas en el ejercicio 1 (quitar palabras vacías, quitar símbolos de puntuación, lematizar con spacy) (20%)

- **Apartado 2**: Utilizar la función de scikit-learn para realizar la transformación a TF-IDF del texto considerando que:
    - Se consideren unigramas, bigramas y trigramas (20%)
    - No se tengan en cuenta los elementos que aparezcan en menos del 5% de los textos (30%)
    - Se utilice el tokenizador incorporado en Scikit-Learn. (10%)
    - Haya un máximo de 200 características. (10%)


> Nota: Como se dije en clase, recordar los valores por defecto de la función. No es una buena práctica duplicar las tareas de preparación de textos. En este ejercicio se está pidiendo realizar el proceso de limpieza **antes** de introducir la cadena de caracteres a la función de scikit-learn, por lo que esta función no debería re-hacer tareas como quitar stopwords, por ejemplo. (la selección correcta de valores de las variables es el último 10% de la valoración del ejercicio).

> Además, recordar una vez más la necesidad de comentar correctamente el ejercicio para obtener la máxima calificación (tengo que saber si teneis claro el proceso que estais llevando a cabo!)


# Apartado 1
## Preprocesar los textos del corpus con las funciones de preprocesado creadas en el ejercicio 1 (quitar palabras vacías, quitar símbolos de puntuación, lematizar con spacy)

Haciendo uso de las funciones creadas en el ejercicio 1, preprocesemos los datos

In [3]:
import pandas as pd
import numpy as np
from tqdm.autonotebook import tqdm
tqdm.pandas()
import nltk

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [8]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('tagsets')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import es_core_news_sm

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Unzipping help/tagsets.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
dataset2 = pd.read_csv("tripadvisor_reviews.tsv", sep = "\t")

Empleamos las mismas funciones que en el ejercicio para preprocesar los datos, sin tener en cuenta todavía la función nltk tpkenizer dado que se incluirá posteriormente en la función de sklearn

In [9]:
def quit_empty_words(text):
  text_tokens = word_tokenize(text)
  text_tokenized = [word for word in text_tokens if not word in stopwords.words('english')]
  return " ".join(text_tokenized)

def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text
nlp = es_core_news_sm.load(disable=['parser', 'ner'])
def lematizar(words):
    mytokens = nlp(words)
    # Lematizamos los tokens y los convertimos  a minusculas
    mytokens = [ word.lemma_ if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    # Extraemos el text en una string
    return mytokens


In [10]:
dataset2['text_processed']= dataset2["Review"].progress_apply(lambda x: quit_empty_words(x))
dataset2['text_processed']= dataset2["text_processed"].apply(remove_punctuations)
dataset2['text_processed']= dataset2["text_processed"].progress_apply(lambda x: lematizar(x))

In [19]:
dataset2["text_processed"] = dataset2["text_processed"].progress_apply(lambda x: " ".join(x))

# Apartado 2
## Utilizar la función de scikit-learn para realizar la transformación a TF-IDF del texto 

De lo que se trata ahora es  de convertir una colección de documentos sin procesar en una matriz de funciones TF-IDF o en el caso de CountVectorizer obterner recuento de tokens. Ambas funciones de sklearn permiten obtener tokens 

In [60]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.feature_extraction import text

In [74]:
tex_pre = dataset2["text_processed"].to_list()

De acuerdo a los parámetros fijados en el enunciado: considerar tanto unigramas, bigramas y trigramas, un máximo de 200 características, sin tener en cuenta las que aparecen en un porcentaje menor al 5%.. etc

In [100]:
tfidf_vect = TfidfVectorizer(lowercase = False, stop_words = None, 
                             ngram_range = (1,3), use_idf = True, smooth_idf = True,
                             norm = "l2",max_features=200, min_df=0.05, tokenizer=None)
data_tfidf= tfidf_vect.fit(tex_pre)

TF-IDF es una medida estadística que evalúa la relevancia de una palabra para un documento en una colección de documentos. Esto se hace multiplicando dos métricas: cuántas veces aparece una palabra en un documento y la frecuencia inversa del documento de la palabra en un conjunto de documentos.
Vemos el número de palabras contenidas en nuestro diccionario con su posición, además de todo el vocabulario contenido en el corpus

In [102]:
print("\n\n TF-IDF VECTORIZER")
print("Obtenemos un dictionario que mapea las palabras con su posición en el vector del vocabulario")
print(data_tfidf.vocabulary_)
print("Obtenemos el vocabulario en si mismo como una lista")
print(data_tfidf.get_feature_names())
print("Numero de características:")
print(len(data_tfidf.get_feature_names()))



 TF-IDF VECTORIZER
Obtenemos un dictionario que mapea las palabras con su posición en el vector del vocabulario
{'nice': 114, 'hotel': 82, 'expensive': 55, 'got': 73, 'good': 72, 'stay': 164, 'arrived': 5, 'evening': 53, 'took': 176, 'reviews': 145, 'check': 29, 'easy': 47, 'little': 94, 'view': 186, 'room': 147, 'clean': 31, 'size': 156, 'bed': 15, 'comfortable': 36, 'like': 93, 'night': 115, 'morning': 110, 'people': 126, 'staying': 166, 'location': 97, 'great': 74, 'walking': 189, 'shopping': 153, 'overall': 124, 'experience': 56, 'pay': 125, 'ok': 120, 'nothing': 118, 'booked': 21, 'suite': 168, 'extra': 57, 'bathroom': 12, 'standard': 161, 'desk': 41, 'said': 149, 'things': 171, 'tv': 181, 'told': 175, 'free': 67, 'breakfast': 22, 'day': 38, 'hard': 77, 'trip': 179, 'staff': 159, 'helpful': 78, 'asked': 7, 'hotels': 83, 'best': 17, 'away': 9, 'know': 90, 'really': 137, 'wanted': 191, 'make': 106, 'sure': 169, 'better': 18, 'left': 92, 'short': 154, 'hours': 85, 'beds': 16, 'room


TF-IDF es muy poderoso para capturar las palabras más importantes en su texto.

In [106]:
df_dict= sorted(data_tfidf.vocabulary_.items(), key=lambda x: x[1], reverse=True)  

In [107]:
df_dict

[('year', 199),
 ('worth', 198),
 ('work', 197),
 ('wonderful', 196),
 ('went', 195),
 ('week', 194),
 ('way', 193),
 ('water', 192),
 ('wanted', 191),
 ('want', 190),
 ('walking', 189),
 ('walk', 188),
 ('visit', 187),
 ('view', 186),
 ('value', 185),
 ('vacation', 184),
 ('used', 183),
 ('usar', 182),
 ('tv', 181),
 ('try', 180),
 ('trip', 179),
 ('towels', 178),
 ('tour', 177),
 ('took', 176),
 ('told', 175),
 ('timar', 174),
 ('thought', 173),
 ('think', 172),
 ('things', 171),
 ('thing', 170),
 ('sure', 169),
 ('suite', 168),
 ('street', 167),
 ('staying', 166),
 ('stayed', 165),
 ('stay', 164),
 ('station', 163),
 ('star', 162),
 ('standard', 161),
 ('staff friendly', 160),
 ('staff', 159),
 ('spacious', 158),
 ('small', 157),
 ('size', 156),
 ('shower', 155),
 ('short', 154),
 ('shopping', 153),
 ('service', 152),
 ('say', 151),
 ('santo', 150),
 ('said', 149),
 ('rooms', 148),
 ('room', 147),
 ('right', 146),
 ('reviews', 145),
 ('returned', 144),
 ('return', 143),
 ('restauran

Visualizamos la salida

Como resultado se puede comprobar que los caracteres alfanumericos estan incluidos dentro de nuestro vector de características. Esto puede deberse a que muchso al ser datos en horas y al eliminar caracteres se hayan juntando formando un token, por ejemplo. Ocurre lo mismo con las unidades de medida, con los apóstrofes, ..etc
Es importante destacar que las palabras más usadas en peso son: **year','worth','work','wonderful','went','week'**. Resulta interesante que dos palabras que podrían ser caracter positivo sean de las importantes, ya que no sabemos el contexto de la frase que lo acompaña. el resto hace referencia claramente a términos relacionados con las vacaciones.